# Unray with auto-scaled grid

In [ ]:
import time
import numpy as np
from unray import *
import pythreejs as three
import ipywidgets
from ipywidgets import jslink
from ipydatawidgets import *
from threeplot import *
from ipyscales import *

## Import heart mesh data and process

Get the mesh data from file, and use it to calculate some dummy scalar fields.

In [ ]:
filename = "../data/heart.npz"
#filename = "../data/brain.npz"
#filename = "../data/aneurysm.npz"

mesh_data = np.load(filename)
cells_array = mesh_data["cells"].astype(np.int32)
points_array = mesh_data["points"].astype(np.float32)

# Coordinates of all vertices in mesh
x = list(points_array.T)  # x[2] = z coordinate array for all vertices

# Model center 3d vector
center = list(map(lambda x: x.mean(), x))

# Model min/max coordinates
bbox = list(map(lambda x: (x.min(), x.max()), x))

# Coordinates with origo shifted to center of model
xm = list(map(lambda x, mp: x - mp, x, center))

# Distance from model center
xd = np.sqrt(sum(map(lambda x: x**2, xm)))
radius = xd.max()

# Distance from center, normalized to max 1.0
func_dist = xd / radius

# A constant for all vertices
func_const = np.ones(x[0].shape)

# x coordinate
func_x = x[0]

# A wave pattern from the center of the model
freq = 4
func_wave = 2.0 + np.sin((freq * 2 * np.pi / radius) * xd)

In [ ]:
cell_midpoints = sum(points_array[cells_array[:,i],:] for i in range(3))
cell_midpoints *= (1.0/3.0)
left_half, = np.where(cell_midpoints[:,0] < 0.5 * (cell_midpoints[:,0].max() - cell_midpoints[:,0].min()))
cell_indicators_array1 = np.zeros(cells_array.shape[0], dtype="int32")
cell_indicators_array1[left_half] = 1
cell_indicators_array2 = 1 - cell_indicators_array1
assert 0 == cells_array.shape[0] - (sum(cell_indicators_array1)+sum(cell_indicators_array2))
print(sum(cell_indicators_array1), sum(cell_indicators_array2))

## Setup data widgets

This will prevent re-upload of data if used multiple times later on, and set up fields and colormaps

In [ ]:
cells = NDArrayWidget(cells_array)
points = NDArrayWidget(points_array)
mesh = Mesh(cells=cells, points=points)

field_values = NDArrayWidget((func_x - func_x.min()) / (func_x.max() - func_x.min()))

field = Field(mesh=mesh, values=field_values)
color_lut = ArrayColorMap(values=[[0,0,0], [1,1,1]])
scalar_lut = ArrayScalarMap(values=[0.2, 0.8])
color_field = ColorField(field=field, lut=color_lut)
scalar_field = ScalarField(field=field, lut=scalar_lut)
plot = None

## A basic plot renderer setup

In [ ]:
def basic_plot_renderer(width, height, plot_group):
    """Set up a renderer object for plotting a group"""
    camera = three.PerspectiveCamera(position=[3, 3, 3], aspect=width/height)
    key_light = three.DirectionalLight(position=[0, 10, 10])
    ambient = three.AmbientLight(intensity=0.5)
    grid = GridCrossWidget(scales=[LinearScaleWidget()]*3, autosize_target=plot_group)
    scene = three.Scene(children=[plot_group, grid, key_light, ambient, camera], background='white')
    controls = three.OrbitControls(camera)
    renderer = three.Renderer(scene, camera, [controls],
                              width=width, height=height)
    return renderer

In [ ]:
plot_scene = three.Group()
renderer = basic_plot_renderer(800, 400, plot_scene)
renderer

## Add unray renderer in x-ray mode

In [ ]:
plot_scene.children = ()

In [ ]:
cell_indicators1 = IndicatorField(mesh=mesh, values=cell_indicators_array1, space="I3")
cell_indicators2 = IndicatorField(mesh=mesh, values=cell_indicators_array2, space="I3")

restrict1 = ScalarIndicators(field=cell_indicators1)
restrict2 = ScalarIndicators(field=cell_indicators2)

plots = [
    SurfacePlot(mesh=mesh, restrict=restrict1),
    XrayPlot(mesh=mesh, restrict=restrict2),
]
plot_scene.add(plots[0])
plot_scene.add(plots[1])

In [ ]:
ip = IsovalueParams()
plot = IsosurfacePlot(mesh=mesh, color=color_field, values=ip)
plot_scene.add(plot)

In [ ]:
box = three.Mesh(three.BoxGeometry(1, 1, 1), three.MeshLambertMaterial(color='red'), position=[15, 0, 0])
plot_scene.add(box)